In [285]:
import os
import re
import csv
import time
import glob
import ctypes
import rasterio
import subprocess
import numpy as np
from functools import partial
import pandas as pd
import geopandas as gpd
from rasterio.merge import merge
from osgeo import ogr, gdal, osr
from multiprocessing import Pool
from rasterio.enums import Resampling

In [286]:
# Global variables
PRODES = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/PRODES/prodes_amazonia_raster_2000_2022_v20231109/prodes_amazonia_raster_2000_2022_v20231109.tif'
ecoregions_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/Ecoregions/Brazilian_Amazon_Ecoregions'
deforested_ecoregions = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/Ecoregions/Deforested_Ecoregions'
ecoregions = [filename for filename in os.listdir(ecoregions_dir) if filename.endswith('.shp')]
APPEARS_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI'
csv_dir = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS/Ecoregion_Analysis.csv'
working_dir = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS'
years = [16, 17, 18, 19, 20, 21, 22]

with open(csv_dir, 'a', newline='') as csv_file: 
    csv_writer = csv.writer(csv_file)

In [287]:
def get_geotransform_from_file(tif_path):
    """
    Get GeoTransform information from a GeoTIFF file.

    Parameters:
    - tif_path: Path to the GeoTIFF file

    Returns:
    - geotransform: GeoTransform information as a tuple
    """
    dataset = gdal.Open(tif_path)
    geotransform = dataset.GetGeoTransform()
    dataset = None  # Close the dataset
    return geotransform

In [288]:
def merge_tiles(year):
    # Input directory containing the TIFF files
    if year < 10:
        input_directory = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles/200{year}_GFC_tiles'
        output_file = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS/200{year}_gfc.tif'
    else:
        input_directory = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles/20{year}_GFC_tiles'
        output_file = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS/20{year}_gfc.tif'

    # Returns if output file already exists
    if os.path.exists(output_file):
        print(f'The GFC output file for year {year} already exists. Aborting.\n')
        return
    
    # Get a list of all TIFF files in the directory
    input_files = glob.glob(os.path.join(input_directory, '*.tif'))

    # Check if there are any TIFF files
    if not input_files:
        print(f'No TIFF files found in the specified directory.\n')
        return

    # Read all input files
    src_files_to_mosaic = [rasterio.open(file) for file in input_files]

    # Merge files
    mosaic, out_trans = merge(src_files_to_mosaic, resampling=Resampling.nearest)

    # Update metadata of the output file
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff", 
        "height": mosaic.shape[1], 
        "width": mosaic.shape[2], 
        "transform": out_trans, 
        "dtype": src_files_to_mosaic[0].dtypes[0],
        "compress": "lzw"
    })
    
    # Write the mosaic to the output file
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

In [289]:
def crop_ecoregion(shp_path, ecoregion, year):
    if year < 10:
        prodes_yearly = f'200{year}_prodes.tif'
        gfc_yearly = f'200{year}_gfc.tif'
        gfc_ecoregion = f'gfc_{ecoregion}_200{year}.tif'
        prodes_ecoregion = f'prodes_{ecoregion}_200{year}.tif'
    else:
        prodes_yearly = f'20{year}_prodes.tif'
        gfc_yearly = f'20{year}_gfc.tif'
        gfc_ecoregion = f'gfc_{ecoregion}_20{year}.tif'
        prodes_ecoregion = f'prodes_{ecoregion}_20{year}.tif'

    prodes_path = os.path.join(deforested_ecoregions, prodes_ecoregion)
    gfc_path = os.path.join(deforested_ecoregions, gfc_ecoregion)

    if os.path.exists(prodes_path):
        print(f'{prodes_ecoregion} already exists.\n')
    else:
        crop_to_ecoregion = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -cutline {shp_path} -crop_to_cutline -dstnodata 255 -of GTiff {prodes_yearly} prodes_temp.tif'
        subprocess.run(crop_to_ecoregion, shell=True)

        lzw_compress = f'gdal_translate -co COMPRESS=LZW prodes_temp.tif {prodes_path}'
        subprocess.run(lzw_compress, shell=True)
        os.remove('prodes_temp.tif')

    if os.path.exists(gfc_path):
        print(f'{gfc_ecoregion} already exists.\n')
    else:
        crop_to_ecoregion = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -cutline {shp_path} -crop_to_cutline -dstnodata 255 -of GTiff {gfc_yearly} gfc_temp.tif'
        subprocess.run(crop_to_ecoregion, shell=True)

        lzw_compress = f'gdal_translate -co COMPRESS=LZW gfc_temp.tif {gfc_path}'
        subprocess.run(lzw_compress, shell=True)
        os.remove('gfc_temp.tif')

    return prodes_path, gfc_path

In [315]:
# Path to shared object (dynamically linked library)
lib = ctypes.CDLL('./libsub_arrays.so')

def analyze_ecoregion(ecoregion):
    # Path to ecoregion shapefile
    shp_path = os.path.join(ecoregions_dir, ecoregion)
    
    for year in years:
        # Creates deforestation data of ecoregion 
        prodes_path, gfc_path = crop_ecoregion(shp_path, ecoregion, year)

        # Extracting geotransform from PRODES deofrestation data
        prodes_geotransform = get_geotransform_from_file(prodes_path)
        prodes_x, prodes_pixel_width, _, prodes_y, _, prodes_pixel_height = prodes_geotransform
        
        # Reading PRODES deforestation data
        with rasterio.open(prodes_path) as prodes:
                prodes_data = prodes.read(1)
        prodes_height, prodes_width = prodes_data.shape
        
        # Reading Global Forest Change (GFC) deforestation data
        with rasterio.open(gfc_path) as gfc:
            gfc_data = gfc.read(1)
        
        # Creating list of APPEARS tifs for the year sorted by day
        if year < 10:
            pattern = f'MOD13Q1.061__250m_16_days_EVI_doy200{year}*.tif'
        else:
            pattern = f'MOD13Q1.061__250m_16_days_EVI_doy20{year}*.tif'   
        tifs = glob.glob(f'{APPEARS_dir}/{pattern}')
        sorted_tifs = sorted(tifs, key=lambda x: int(x.split("_doy")[1][:7]))
        
        # Extracting APPEARS tif geotransform 
        appears_geotransform = get_geotransform_from_file (sorted_tifs[0])
        appears_x, appears_pixel_width, _, appears_y, _, appears_pixel_height = appears_geotransform
        
        # Calculate the pixel index for the top-left coordinate of the PRODES tif within the APPEARS tif
        prodes_pixel_index_x = int((prodes_x - appears_geotransform[0]) / appears_pixel_width)
        prodes_pixel_index_y = int((prodes_y - appears_geotransform[3]) / appears_pixel_height)

        # Use the pixel indices to calculate the latitude and longitude in the APPEARS tif
        prodes_lat = appears_geotransform[3] + prodes_pixel_index_y * appears_geotransform[5]
        prodes_lon = appears_geotransform[0] + prodes_pixel_index_x * appears_geotransform[1]
        
        print(f'Index within APPEARS: {prodes_pixel_index_x}\n')
        print(f'Coordinates of that index: {prodes_lat}, {prodes_lon}\n')
        print(f'Top left coordinates of original prodes: {prodes_y}, {prodes_x}\n')
        
        # Calculate it here!
        
        # Calculating pixel coordinates
        # appears_top_left_x = int((prodes_x - appears_x) / appears_pixel_width)
        # appears_top_left_y = int((prodes_y - appears_y) / appears_pixel_height)
        
        
        
        day = 1
        # Boolean to check if an error occured reading the previous tif file
        error = False
        for i, image in enumerate(sorted_tifs): # Processes all APPEARS tif files
            
            # ONLY FOR TESTING
            break
            
            try: # Reading image
                with rasterio.open(image) as appears:
                    appears_data = appears.read(1)
                
                # Sets prev if it is day 1 or an error occured reading the previous tif file
                if day == 1 or not prev or error: 
                    print('getting appears slice')
                    
                    
            except Exception as e: # Error with image
                print(f"Error reading data from {path}: {e}")
                # Continue to the next iteration
            
            day += 16
            
    print(f'Processed {ecoregion}!')
            

In [316]:
if __name__ == '__main__':
    
    for year in years:
        merge_tiles(year)
        
        if year < 10:
            prodes_interm = f'200{year}_prodes_interm.tif'
            prodes_yearly = f'200{year}_prodes.tif'
        else:
            prodes_interm = f'20{year}_prodes_interm.tif'
            prodes_yearly = f'20{year}_prodes.tif'
            
        if os.path.exists(os.path.join(working_dir, prodes_yearly)):
            print(f'{prodes_yearly} already exists.\n')
        else:
            # Create prodes_yearly raster if it does not already exist
            extract_year = f'gdal_calc.py -A {PRODES} --outfile={prodes_interm} --calc="A=={year}" --NoDataValue=255'
            subprocess.run(extract_year, shell=True)
            
            lzw_compress = f'gdal_translate -co COMPRESS=LZW {prodes_interm} {prodes_yearly}'
            subprocess.run(lzw_compress, shell=True)
            
            os.remove(prodes_interm)
            
    count = 0
    for ecoregion in ecoregions:
        if count == 3:
            break
        start = time.time()
        analyze_ecoregion(ecoregion)
        end = time.time()
        print(f'Total time to process {ecoregion}: {end - start}')
        count += 1
            
    '''
    with Pool() as pool:
        pool.map(analyze_ecoregions, ecoregions)
        pool.join
    '''        
         
    print('\nFinished!\n')

The GFC output file for year 16 already exists. Aborting.

2016_prodes.tif already exists.

The GFC output file for year 17 already exists. Aborting.

2017_prodes.tif already exists.

The GFC output file for year 18 already exists. Aborting.

2018_prodes.tif already exists.

The GFC output file for year 19 already exists. Aborting.

2019_prodes.tif already exists.

The GFC output file for year 20 already exists. Aborting.

2020_prodes.tif already exists.

The GFC output file for year 21 already exists. Aborting.

2021_prodes.tif already exists.

The GFC output file for year 22 already exists. Aborting.

2022_prodes.tif already exists.

prodes_Mato_Grosso_seasonal_forests.shp_2016.tif already exists.

gfc_Mato_Grosso_seasonal_forests.shp_2016.tif already exists.

Index within APPEARS: 7020

Coordinates of that index: -5.2020833328673035, -59.3583333280157

Top left coordinates of original prodes: -5.20221408717634, -59.35693748205469

prodes_Mato_Grosso_seasonal_forests.shp_2017.tif alr

In [ ]:
'''
path = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2017129_aid0001.tif'

with rasterio.open(path) as appears:
    appears_data = appears.read(1)
print(appears_data)